In [1]:
import torch
import matplotlib.pyplot as plt
import yousef_model as model
import Naicheng.res.process_data.process_output as pro_out
import Naicheng.res.process_data.process_raw_data as pr
import Naicheng.res.process_data.dire_and_coor as dc
import Naicheng.res.process_data.loop_overlap as lp
from Naicheng.res.process_data.dataset import tensor_dataset
from torch.utils.data import DataLoader
import seaborn as sns
import numpy as np
import json

1. read raw data
2. calculate the n to n distance of input polymer

In [2]:
input_coorinate = pr.read_coordinate(16, '../../data/Coordinates.dat')
input_three_dirc = dc.coordinates_directions_three(input_coorinate)

In [3]:
epoch = 4000
name = f'models/yousef_gan_{epoch}.pth.tar'
gen = model.GeneratorNet()
check_point = torch.load(name)
gen.load_state_dict(check_point['gen_state_dict'])
noise = model.noise(8)
output = gen(noise).data.numpy()
output_direction= pro_out.slice_directions(output)
output_coor = dc.direction_coordinate_three(output_direction)
num_ite = 2000
output_direction = np.zeros([num_ite * 8, 15, 1])
for j in range(num_ite):
    # get output
    noise = model.noise(8)
    output = gen(noise).data.numpy()
    # slice the output to a (n ,15 ,1)
    output_direction[8 * j : 8 * (j + 1)] = pro_out.slice_directions(output)
output_coor = dc.direction_coordinate_three(output_direction)

Remove duplicated

In [4]:
remove_str, remove_index = lp.remove_duplicated(output_direction)

In [5]:
unique_coor, unique_direction = lp.get_unique_coor(remove_index, output_coor, output_direction)

In [25]:
overlap = lp.check_overlap(input_three_dirc ,unique_direction)
print (overlap)

7817


get the length of loop

In [7]:
loop_all = []
for i in range(len(unique_coor)):
    if lp.check_fold_cross(unique_coor[i]):
        loop_all += lp.loop_length(unique_coor[i])

In [8]:
a = {}
for i in loop_all:
    if loop_all.count(i)>1:
        a[i] = loop_all.count(i)

In [9]:
print (f'ratio of loop with length 4 is {a[4] / len(unique_coor)}\n'
       f'ratio of loop with length 6 is {a[6]/ len(unique_coor)}\n'
       f'ratio of loop with length 8 is {a[8]/ len(unique_coor)}\n'
       f'ratio of loop with length 10 is {a[10]/ len(unique_coor)}\n'
       f'ratio of loop with length 12 is {a[12]/ len(unique_coor)}\n'
       f'ratio of loop with length 14 is {a[14]/ len(unique_coor)}\n')

ratio of loop with length 4 is 0.15072483176537546
ratio of loop with length 6 is 0.15447453881951187
ratio of loop with length 8 is 0.12698784693160134
ratio of loop with length 10 is 0.0835649000636111
ratio of loop with length 12 is 0.0504257479884384
ratio of loop with length 14 is 0.02325153168837256



Get the rg2, rg4 and rg6 for unique, self-avoiding generating polymer

In [10]:
import Naicheng.res.process_data.rg2 as rg
fold, cross, self_avoid = lp.count_fold_cross(unique_coor)
self_avoid_polymers = lp.arrange_self_avoid_polymer(unique_coor, self_avoid)
overlap = lp.check_overlap(input_three_dirc ,self_avoid_polymers)

ValueError: setting an array element with a sequence.

In [11]:
rg2, rg4, rg6 = rg.rg2s(self_avoid_polymers)

In [14]:
rg2_input, rg4_input, rg6_input = rg.rg2s(input_coorinate)

In [27]:
print (f'generate {len(output_coor)} polymers \n'
       f'{len(unique_coor)} polymers are unique \n'
       f'unique ratio is {len(unique_coor) / len(output_coor)}\n'
       f'{len(self_avoid_polymers)} polymers are self avoid, percentage is {len(self_avoid_polymers)/ len(unique_coor)}'
       f'{overlap} is overlap with input polymer \n'
       f'overlap percentage is {overlap / len(self_avoid_polymers) * 100}% \n'
       f'rg2 is {rg2}, ratio = {rg2/rg2_input},\n'
       f'rg4 is {rg4}, ratio = {rg4/rg4_input}\n'
       f'rg6 is {rg6}, ratio = {rg6/rg6_input}')

generate 230000 polymers 
179214 polymers are unique 
unique ratio is 0.7791913043478261
127632 polymers are self avoid, percentage is 0.71217650406776267817 is overlap with input polymer 
overlap percentage is 6.1246395888178515% 
rg2 is 7.653341124287013, ratio = 1.1263744794444965,
rg4 is 104.16398399097662, ratio = 1.262967284962581
rg6 is 1914.5879405084313, ratio = 1.3995039937471718


1. generate 113000 polymers
2. remove duplications from generating polymer, remain , the duplication ratio is
3. for unique polymers, count loops with different lengths
4. remove polymers contains loops(remain self avoid polymers)
5. for unique, self avoid generating polymers, check the ratio of overlaping with input data

